In [3]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests

pd.options.display.max_columns = None

In [4]:
datos = ['https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=60',
        'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=120', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=180',
         'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=240', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=300',
         'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=360', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=420']

In [5]:
jugar = pd.DataFrame()

for i in datos:
    url = i
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    html.status_code
    
    productos = soup.find_all("td",{"class": "col col-ae"})
    
    edades = {'edad': []}
    
    for x in productos:
        edades['edad'].append(x)
        
    jedad = pd.DataFrame(edades)
    jedad["edad"] = jedad['edad'].astype(str)
    jedad['edad'] = jedad['edad'].str.extract(r'(?<=\"\>)(.*)(?=\<\/td\>)')
    
    jugar = pd.concat([jugar, jedad], axis=0)

In [6]:
jugar = jugar.reset_index(drop=True)

In [7]:
pais = pd.DataFrame()

for i in datos:
    url = i
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    html.status_code
    
    pais_nac = {'pais': []}
    
    productos2 = soup.find_all("td",{"class": "col-name"})
    
    for x in productos2:
        pais_nac['pais'].append(x.find('img').get('title')) 
        
    pnac = pd.DataFrame(pais_nac)
    
    paisnacimient = pnac.dropna()
    paisnacimient = paisnacimient.reset_index(drop=True)
    
    pais = pd.concat([pais, paisnacimient], axis=0)

In [8]:
pais = pais.reset_index(drop=True)

In [9]:
pais.head(2)

,pais
0,Uruguay
1,Morocco


In [10]:
pais_edad = pd.concat([jugar, pais], axis=1)

In [11]:
pais_edad = pais_edad.drop(pais_edad.index[[31,80,89,95,116,158,208,263]])
pais_edad = pais_edad.reset_index(drop=True)

In [12]:
pais_edad.head(2)

,edad,pais
0,21,Uruguay
1,19,Morocco


In [14]:
pais_edad.to_csv('../../data/precio/paisedad_jugadores.csv')